In [3]:
using Revise
using BenchmarkTools

In [4]:
includet("measure_mc.jl")

In [8]:
const HeisenbergSpin = Tuple{Float64,Float64,Float64}

function compute_mq(q,kagome,spins,triangles)

    @assert length(keys(kagome)) == length(spins)

    sq = fill((0.0+0im,0.0+0im,0.0+0im),3)
    for i in triangles
        for j in 1:3
            idx = i[j]
            R = kagome[idx]
            sq[j] = sq[j] .+ spins[idx].*exp((q⋅R)*im)
        end
    end

    ss = 0.

    for i in 1:3
        ss += sq[i] ⋅ sq[i]
    end

    return real(ss)/(3*length(triangles)^2)
end

function compute_mq_fast(q,kagome,spins,triangles)
    @assert length(keys(kagome)) == length(spins)

    sq = SVector{3,Complex{Float64}}(0.0, 0.0, 0.0)
    for i in triangles
        for j in 1:3
            idx = i[j]
            R = kagome[idx]
            sq[j] = sq[j] .+ spins[idx].*exp((q⋅R)*im)
        end
    end

    ss = 0.

    for i in 1:3
        ss += sq[i] ⋅ sq[i]
    end

    return real(ss)/(3*length(triangles)^2)
end

function mk_kagome(L)

    kagome = Dict()
    idx = 1
    L = 2L

    for (i,j) in Iterators.product(1:L,1:L)
        mod_site = mod.((i,j),L)
        if mod(mod_site[1],2) == 0 && mod(mod_site[2],2) == 1
            continue
        end

        kagome[idx] = copy.(mod_site)
        idx += 1
    end

    return kagome
end

mk_kagome (generic function with 1 method)

In [18]:
L = 12
num_spins = 3L^2
num_triangles = L^2

triangles = fill((1,1,1), num_triangles)
spins = fill((0.0, 0.0, 0.0),num_spins)
kagome = mk_kagome(Int(sqrt(num_spins/3)))

kagome_array = [kagome[i] for i in 1:num_spins]

@benchmark compute_mq((0.,0.), kagome_array, spins, triangles)

BenchmarkTools.Trial: 
  memory estimate:  240 bytes
  allocs estimate:  2
  --------------
  minimum time:     3.486 μs (0.00% GC)
  median time:      3.724 μs (0.00% GC)
  mean time:        4.185 μs (0.00% GC)
  maximum time:     24.061 μs (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     8

In [46]:
#using StaticArrays
function mycross(s1, s2)
    s1[1]s2[2] - s1[2]s2[1]
end

function compute_vector_chirality2(spins::AbstractArray{Float64,2},
                                  triangles::Vector{Tuple{Int64,Int64,Int64}})

    vc = 0.0
    num_spins = size(spins)[2]
    num_triangles = length(triangles)
    #MVector{3,Float64}
    @assert num_spins == 3num_triangles
    for i in triangles
        for j in 1:3
            s1 = view(spins, :, i[j])
            s2 = view(spins, :, i[ifelse(j==3,1,j+1)])
            #s1 = spins[:, i[j]]
            #s2 = spins[:, i[ifelse(j==3,1,j+1)]]
            vc += mycross(s1, s2)
        end
    end
    return vc / num_spins
end

compute_vector_chirality2 (generic function with 2 methods)

In [32]:
L = 12
num_spins = 3L^2
num_triangles = L^2

triangles = fill((1,1,1), num_triangles)
spins = fill(0.0, 3, num_spins)

@benchmark compute_vector_chirality2(spins, triangles)

BenchmarkTools.Trial: 
  memory estimate:  94.50 KiB
  allocs estimate:  864
  --------------
  minimum time:     27.752 μs (0.00% GC)
  median time:      29.154 μs (0.00% GC)
  mean time:        33.818 μs (3.81% GC)
  maximum time:     901.331 μs (90.06% GC)
  --------------
  samples:          10000
  evals/sample:     1

In [35]:
x = fill(0, 10)
x[1:2] .= 1
x

10-element Array{Int64,1}:
 1
 1
 0
 0
 0
 0
 0
 0
 0
 0

In [37]:
function test_f!(x)
    x .= 1
end
x = fill(0, 10)
test_f!(x[1:2])
x

10-element Array{Int64,1}:
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0

In [38]:
x = fill(0, 10)
test_f!(view(x, 1:2))
x

10-element Array{Int64,1}:
 1
 1
 0
 0
 0
 0
 0
 0
 0
 0

In [39]:
view(x, 1:2)

2-element view(::Array{Int64,1}, 1:2) with eltype Int64:
 1
 1

In [42]:
x = fill(0, 10)
test_f!(@view x[1:2])
x

10-element Array{Int64,1}:
 1
 1
 0
 0
 0
 0
 0
 0
 0
 0

In [44]:
num_temps = 2
x = fill(0.0, 3, num_spins, num_temps)
x2 = [fill(0.0, 3, num_spins) for i in 1:num_temps]

2-element Array{Array{Float64,2},1}:
 [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]
 [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

In [47]:
compute_vector_chirality2(view(spins, :, :, 1), triangles)

0.0

In [30]:
@code_warntype compute_vector_chirality2(spins, triangles)

Variables
  #self#::Core.Compiler.Const(compute_vector_chirality2, false)
  spins::Array{Float64,2}
  triangles::Array{Tuple{Int64,Int64,Int64},1}
  vc::Float64
  num_spins::Int64
  num_triangles::Int64
  @_7::Union{Nothing, Tuple{Tuple{Int64,Int64,Int64},Int64}}
  i::Tuple{Int64,Int64,Int64}
  @_9::Union{Nothing, Tuple{Int64,Int64}}
  j::Int64
  s1::SubArray{Float64,1,Array{Float64,2},Tuple{Base.Slice{Base.OneTo{Int64}},Int64},true}
  s2::SubArray{Float64,1,Array{Float64,2},Tuple{Base.Slice{Base.OneTo{Int64}},Int64},true}

Body::Float64
1 ──       Core.NewvarNode(:(@_7))
│          (vc = 0.0)
│    %3  = Main.size(spins)::Tuple{Int64,Int64}
│          (num_spins = Base.getindex(%3, 2))
│          (num_triangles = Main.length(triangles))
│    %6  = num_spins::Int64
│    %7  = (3 * num_triangles)::Int64
│    %8  = (%6 == %7)::Bool
└───       goto #3 if not %8
2 ──       goto #4
3 ── %11 = Base.AssertionError("num_spins == 3num_triangles")::AssertionError
└───       Base.throw(%11)
4 ┄─ %

In [19]:
function convert_spins_to_array(spins::Vector{HeisenbergSpin})
    N = length(spins)
    spins_array = Array{Float64,2}(undef, 3, N)
    for i in 1:N, j=1:3
        spins_array[j,i] = spins[i][j]
    end
    spins_array
end

convert_spins_to_array (generic function with 1 method)

In [1]:
@benchmark convert_spins_to_array(spins)

LoadError: UndefVarError: @benchmark not defined

In [14]:
L = 12
num_spins = 3L^2
num_triangles = L^2

triangles = fill((1,1,1), num_triangles)
spins = fill((0.0, 0.0, 0.0), num_spins)

@benchmark compute_vector_chirality(spins, triangles)

BenchmarkTools.Trial: 
  memory estimate:  50.75 KiB
  allocs estimate:  433
  --------------
  minimum time:     12.756 μs (0.00% GC)
  median time:      15.710 μs (0.00% GC)
  mean time:        16.712 μs (4.05% GC)
  maximum time:     720.229 μs (96.25% GC)
  --------------
  samples:          10000
  evals/sample:     1

In [12]:
@code_warntype compute_vector_chirality(spins, triangles)

Variables
  #self#::Core.Compiler.Const(compute_vector_chirality, false)
  spins::Array{Tuple{Float64,Float64,Float64},1}
  triangles::Array{Tuple{Int64,Int64,Int64},1}
  vc::Float64
  num_spins::Int64
  num_triangles::Int64
  spins2::Array{Array{Float64,1},1}
  @_8::Union{Nothing, Tuple{Tuple{Int64,Int64,Int64},Int64}}
  i::Tuple{Int64,Int64,Int64}
  @_10::Union{Nothing, Tuple{Int64,Int64}}
  j::Int64
  s1::Array{Float64,1}
  s2::Array{Float64,1}

Body::Float64
1 ──       Core.NewvarNode(:(spins2))
│          Core.NewvarNode(:(@_8))
│          (vc = 0.0)
│          (num_spins = Main.length(spins))
│          (num_triangles = Main.length(triangles))
│    %6  = num_spins::Int64
│    %7  = (3 * num_triangles)::Int64
│    %8  = (%6 == %7)::Bool
└───       goto #3 if not %8
2 ──       goto #4
3 ── %11 = Base.AssertionError("num_spins == 3num_triangles")::AssertionError
└───       Base.throw(%11)
4 ┄─ %13 = Base.broadcasted(Main.collect, spins)::Base.Broadcast.Broadcasted{Base.Broadcast.Def